In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import os
from skimage.transform import rotate
import pandas as pd
import matplotlib.pyplot as plt

# Define global constants
IMG_HEIGHT = 64
IMG_WIDTH = 64
EPOCHS = 10
BATCH_SIZE = 32
learning_rates = [0.01, 0.005, 0.001]

# Initialize result dictionaries
original_data_results = {}
augmented_data_results = {}

# --- Data Loading and Preprocessing (from previous steps) ---
# Define the base directory where content was extracted
base_dir = '/content/cat_dog/'

# Define paths for training and testing images and labels
train_images_dir = os.path.join(base_dir, 'train')
train_labels_file = os.path.join(base_dir, 'train_class_labels.txt')
test_images_dir = os.path.join(base_dir, 'test')
test_labels_file = os.path.join(base_dir, 'test_class_labels.txt')

def load_images_from_directory(directory, target_size=(IMG_HEIGHT, IMG_WIDTH)):
    images = []
    image_filenames = sorted([f for f in os.listdir(directory) if f.endswith(('.jpg', '.png'))])
    for filename in image_filenames:
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path) # Reads image in BGR format
        if img is not None:
            img = cv2.resize(img, target_size) # Resize image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR to RGB
            images.append(img)
    return images

def load_labels_from_file(filepath):
    labels = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            labels.append(int(parts[-1]))
    return labels

# Load training data
train_images = load_images_from_directory(train_images_dir)
train_labels = load_labels_from_file(train_labels_file)

# Load testing data
test_images = load_images_from_directory(test_images_dir)
test_labels = load_labels_from_file(test_labels_file)

# Convert to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize pixel values to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# --- Custom Data Augmentation Function (from previous steps) ---
def augment_image_rotation(image, label):
    augmented_images = []
    augmented_labels = []

    angle1 = np.random.uniform(-45, 45)
    angle2 = np.random.uniform(-45, 45)

    rotated_image1 = rotate(image, angle=angle1, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image1)
    augmented_labels.append(label)

    rotated_image2 = rotate(image, angle=angle2, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image2)
    augmented_labels.append(label)

    return augmented_images, augmented_labels

# --- Generate Augmented Training Dataset (from previous steps) ---
augmented_train_images_list = []
augmented_train_labels_list = []

for i in range(len(train_images)):
    image = train_images[i]
    label = train_labels[i]

    aug_imgs, aug_labels = augment_image_rotation(image, label)

    augmented_train_images_list.extend(aug_imgs);
    augmented_train_labels_list.extend(aug_labels);

augmented_train_images = np.array(augmented_train_images_list)
augmented_train_labels = np.array(augmented_train_labels_list)

combined_augmented_train_images = np.concatenate((train_images, augmented_train_images), axis=0)
combined_augmented_train_labels = np.concatenate((train_labels, augmented_train_labels), axis=0)

# --- Define CNN Model Template (from previous steps) ---
def create_cnn_model(input_shape):
    model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.3),

        # Flatten the output for the Dense layers
        Flatten(),

        # Dense layers
        Dense(100, activation='relu'),
        Dropout(0.5),

        # Output layer for binary classification
        Dense(1, activation='sigmoid')
    ])
    return model

print("Environment and data initialized successfully.")
print(f"train_images shape: {train_images.shape}, train_labels shape: {train_labels.shape}")
print(f"test_images shape: {test_images.shape}, test_labels shape: {test_labels.shape}")
print(f"combined_augmented_train_images shape: {combined_augmented_train_images.shape}")
print(f"combined_augmented_train_labels shape: {combined_augmented_train_labels.shape}")

# --- Training and Evaluation Loop for Original Data ---
print("\n--- Training on Original Data ---")
for lr in learning_rates:
    print(f"\nTraining model with learning rate: {lr}")

    model_original_lr = create_cnn_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    optimizer = Adam(learning_rate=lr)
    model_original_lr.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    history = model_original_lr.fit(
        train_images,
        train_labels,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=0
    )

    test_loss, test_accuracy = model_original_lr.evaluate(test_images, test_labels, verbose=0)
    print(f"Learning Rate: {lr}, Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")

    original_data_results[lr] = {
        'history': history.history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy
    }
print("Finished training and evaluation for all specified learning rates on original data.")

# --- Training and Evaluation Loop for Augmented Data ---
print("\n--- Training on Augmented Data ---")
for lr in learning_rates:
    print(f"\nTraining model with learning rate: {lr} on augmented data")

    model_augmented_lr = create_cnn_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    optimizer = Adam(learning_rate=lr)
    model_augmented_lr.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    history = model_augmented_lr.fit(
        combined_augmented_train_images,
        combined_augmented_train_labels,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=0
    )

    test_loss, test_accuracy = model_augmented_lr.evaluate(test_images, test_labels, verbose=0)
    print(f"Learning Rate: {lr}, Test Accuracy (Augmented Data): {test_accuracy:.4f}, Test Loss (Augmented Data): {test_loss:.4f}")

    augmented_data_results[lr] = {
        'history': history.history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy
    }
print("Finished training and evaluation for all specified learning rates on augmented data.")

# --- Summary Table Generation ---
summary_data = []

# Process original data results
for lr in learning_rates:
    result = original_data_results[lr]
    summary_data.append({
        'Learning Rate': lr,
        'Data Type': 'Original',
        'Test Accuracy': result['test_accuracy'],
        'Test Loss': result['test_loss']
    })

# Process augmented data results
for lr in learning_rates:
    result = augmented_data_results[lr]
    summary_data.append({
        'Learning Rate': lr,
        'Data Type': 'Augmented',
        'Test Accuracy': result['test_accuracy'],
        'Test Loss': result['test_loss']
    })

summary_df = pd.DataFrame(summary_data)
print("\nSummary of Model Performance:")
print(summary_df.to_string(index=False))


# --- Plotting the optimization history ---

def plot_history(history, title):
    plt.figure(figsize=(12, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history['accuracy'], label='Training Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Find the best learning rate for original data (based on test accuracy)
best_lr_original = max(original_data_results, key=lambda lr: original_data_results[lr]['test_accuracy'])
best_history_original = original_data_results[best_lr_original]['history']
plot_history(best_history_original, f'Original Data (LR: {best_lr_original})')

# Find the best learning rate for augmented data (based on test accuracy)
best_lr_augmented = max(augmented_data_results, key=lambda lr: augmented_data_results[lr]['test_accuracy'])
best_history_augmented = augmented_data_results[best_lr_augmented]['history']
plot_history(best_history_augmented, f'Augmented Data (LR: {best_lr_augmented})')

print("Optimization histories plotted for the best learning rates.")

Environment and data initialized successfully.
train_images shape: (10000, 64, 64, 3), train_labels shape: (10000,)
test_images shape: (2000, 64, 64, 3), test_labels shape: (2000,)
combined_augmented_train_images shape: (30000, 64, 64, 3)
combined_augmented_train_labels shape: (30000,)

--- Training on Original Data ---

Training model with learning rate: 0.01


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Learning Rate: 0.01, Test Accuracy: 0.5000, Test Loss: 0.7235

Training model with learning rate: 0.005
Learning Rate: 0.005, Test Accuracy: 0.5000, Test Loss: 0.7250

Training model with learning rate: 0.001


# Task
Unzip `cat_dog.zip`, load and preprocess the images, implement a custom data augmentation function to rotate images, and then train and evaluate a Convolutional Neural Network (CNN) model for image classification. Compare the model's performance when trained on original data versus augmented data, using different learning rates, summarize the results in a table, plot optimization histories, and analyze the impact of data augmentation and learning rates on model performance.

## Extract and Load Data

### Subtask:
Unzip the `cat_dog.zip` file, then load all training and testing images and their corresponding labels from the extracted folders and `.txt` files. Preprocess the images by normalizing pixel values to a [0, 1] range.


**Reasoning**:
The first step is to unzip the `cat_dog.zip` file as instructed, so I will import the `zipfile` module and use it to extract the contents.



In [ ]:
import zipfile
import os

# Define the path to the zip file
zip_file_path = '/content/cat_dog.zip'

# Define the directory where contents will be extracted
extract_dir = '/content/'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"'{zip_file_path}' unzipped to '{extract_dir}' successfully.")

# List the contents of the extracted directory to verify
print("Contents of the extracted directory:")
print(os.listdir(extract_dir + 'cat_dog'))

'/content/cat_dog.zip' unzipped to '/content/' successfully.
Contents of the extracted directory:
['test_class_labels.txt', 'train', 'test', 'train_class_labels.txt']


**Reasoning**:
Now that the data is unzipped, I need to load the images and their corresponding labels. This involves importing necessary libraries like `cv2` and `numpy`, defining paths, and creating helper functions to load images from directories and labels from text files as per the instructions.



In [ ]:
import cv2
import numpy as np

# Define the base directory where content was extracted
base_dir = '/content/cat_dog/'

# Define paths for training and testing images and labels
train_images_dir = os.path.join(base_dir, 'train')
train_labels_file = os.path.join(base_dir, 'train_class_labels.txt')
test_images_dir = os.path.join(base_dir, 'test')
test_labels_file = os.path.join(base_dir, 'test_class_labels.txt')

# Define image dimensions
IMG_HEIGHT = 64
IMG_WIDTH = 64

def load_images_from_directory(directory, target_size=(IMG_HEIGHT, IMG_WIDTH)):
    images = []
    image_filenames = sorted([f for f in os.listdir(directory) if f.endswith(('.jpg', '.png'))])
    for filename in image_filenames:
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path) # Reads image in BGR format
        if img is not None:
            img = cv2.resize(img, target_size) # Resize image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR to RGB
            images.append(img)
    print(f"Loaded {len(images)} images from {directory}")
    return images

def load_labels_from_file(filepath):
    labels = []
    with open(filepath, 'r') as f:
        for line in f:
            # Assuming each line is just the label, and it needs to be an integer
            labels.append(int(line.strip()))
    print(f"Loaded {len(labels)} labels from {filepath}")
    return labels

# Load training data
train_images = load_images_from_directory(train_images_dir)
train_labels = load_labels_from_file(train_labels_file)

# Load testing data
test_images = load_images_from_directory(test_images_dir)
test_labels = load_labels_from_file(test_labels_file)

# Convert to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize pixel values to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

print(f"Training images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"Training labels shape: {train_labels.shape}, dtype: {train_labels.dtype}")
print(f"Testing images shape: {test_images.shape}, dtype: {test_images.dtype}")
print(f"Testing labels shape: {test_labels.shape}, dtype: {test_labels.dtype}")


Loaded 10000 images from /content/cat_dog/train


ValueError: invalid literal for int() with base 10: '9_cat.png\t0'

**Reasoning**:
The previous code failed because the label file contains both the image filename and the label on each line, separated by a tab, which caused a `ValueError` when trying to convert the whole line to an integer. I will modify the `load_labels_from_file` function to split each line by the tab character and extract only the integer label.



In [ ]:
import cv2
import numpy as np

# Define the base directory where content was extracted
base_dir = '/content/cat_dog/'

# Define paths for training and testing images and labels
train_images_dir = os.path.join(base_dir, 'train')
train_labels_file = os.path.join(base_dir, 'train_class_labels.txt')
test_images_dir = os.path.join(base_dir, 'test')
test_labels_file = os.path.join(base_dir, 'test_class_labels.txt')

# Define image dimensions
IMG_HEIGHT = 64
IMG_WIDTH = 64

def load_images_from_directory(directory, target_size=(IMG_HEIGHT, IMG_WIDTH)):
    images = []
    image_filenames = sorted([f for f in os.listdir(directory) if f.endswith(('.jpg', '.png'))])
    for filename in image_filenames:
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path) # Reads image in BGR format
        if img is not None:
            img = cv2.resize(img, target_size) # Resize image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR to RGB
            images.append(img)
    print(f"Loaded {len(images)} images from {directory}")
    return images

def load_labels_from_file(filepath):
    labels = []
    with open(filepath, 'r') as f:
        for line in f:
            # Split the line by tab and take the last element (the label)
            parts = line.strip().split('\t')
            labels.append(int(parts[-1]))
    print(f"Loaded {len(labels)} labels from {filepath}")
    return labels

# Load training data
train_images = load_images_from_directory(train_images_dir)
train_labels = load_labels_from_file(train_labels_file)

# Load testing data
test_images = load_images_from_directory(test_images_dir)
test_labels = load_labels_from_file(test_labels_file)

# Convert to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize pixel values to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

print(f"Training images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"Training labels shape: {train_labels.shape}, dtype: {train_labels.dtype}")
print(f"Testing images shape: {test_images.shape}, dtype: {test_images.dtype}")
print(f"Testing labels shape: {test_labels.shape}, dtype: {test_labels.dtype}")


Loaded 10000 images from /content/cat_dog/train
Loaded 10000 labels from /content/cat_dog/train_class_labels.txt
Loaded 2000 images from /content/cat_dog/test
Loaded 2000 labels from /content/cat_dog/test_class_labels.txt
Training images shape: (10000, 64, 64, 3), dtype: float64
Training labels shape: (10000,), dtype: int64
Testing images shape: (2000, 64, 64, 3), dtype: float64
Testing labels shape: (2000,), dtype: int64


## Implement Custom Data Augmentation

### Subtask:
Create a function that takes an image and its label, and generates two additional images by rotating the original image randomly between -45 and 45 degrees, using `skimage.transform.rotate`. These augmented images will retain the original image's class label.


**Reasoning**:
I need to implement the `augment_image_rotation` function as described in the subtask. This involves importing necessary libraries, defining the function, generating random rotation angles, applying `skimage.transform.rotate`, and returning the augmented images along with their labels.



In [ ]:
import numpy as np
from skimage.transform import rotate

def augment_image_rotation(image, label):
    augmented_images = []
    augmented_labels = []

    # Generate two random angles between -45 and 45 degrees
    angle1 = np.random.uniform(-45, 45)
    angle2 = np.random.uniform(-45, 45)

    # Rotate the image using the first angle
    # preserve_range=True is important to keep pixel values in [0, 1] range for normalized images
    # mode='edge' fills in new pixels with the nearest border pixels
    rotated_image1 = rotate(image, angle=angle1, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image1)
    augmented_labels.append(label)

    # Rotate the image using the second angle
    rotated_image2 = rotate(image, angle=angle2, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image2)
    augmented_labels.append(label)

    return augmented_images, augmented_labels

print("Defined 'augment_image_rotation' function for custom data augmentation.")

Defined 'augment_image_rotation' function for custom data augmentation.


## Generate Augmented Training Dataset

### Subtask:
Apply the custom data augmentation function to each image in the original training set. Combine the original training images and the newly generated augmented images, along with their labels, to form the complete augmented training dataset.


**Reasoning**:
I need to apply the custom data augmentation function to the original training dataset. This involves iterating through the training images, applying the `augment_image_rotation` function, collecting the augmented images and labels, and then combining them with the original training data to form a comprehensive augmented training dataset. This step directly addresses the subtask's requirements for generating an augmented training dataset.



In [ ]:
augmented_train_images_list = []
augmented_train_labels_list = []

# Iterate through each original training image and its label
for i in range(len(train_images)):
    image = train_images[i]
    label = train_labels[i]

    # Generate two augmented images and their labels
    aug_imgs, aug_labels = augment_image_rotation(image, label)

    # Extend the lists with the augmented data
    augmented_train_images_list.extend(aug_imgs);
    augmented_train_labels_list.extend(aug_labels);

# Convert augmented lists to NumPy arrays
augmented_train_images = np.array(augmented_train_images_list)
augmented_train_labels = np.array(augmented_train_labels_list)

# Concatenate original training data with augmented training data
combined_augmented_train_images = np.concatenate((train_images, augmented_train_images), axis=0)
combined_augmented_train_labels = np.concatenate((train_labels, augmented_train_labels), axis=0)

print(f"Original training images shape: {train_images.shape}")
print(f"Augmented images shape: {augmented_train_images.shape}")
print(f"Combined augmented training images shape: {combined_augmented_train_images.shape}")
print(f"Original training labels shape: {train_labels.shape}")
print(f"Augmented labels shape: {augmented_train_labels.shape}")
print(f"Combined augmented training labels shape: {combined_augmented_train_labels.shape}")

Original training images shape: (10000, 64, 64, 3)
Augmented images shape: (20000, 64, 64, 3)
Combined augmented training images shape: (30000, 64, 64, 3)
Original training labels shape: (10000,)
Augmented labels shape: (20000,)
Combined augmented training labels shape: (30000,)


## Define CNN Model

### Subtask:
Construct the Convolutional Neural Network (CNN) model as per the provided architecture specifications, including Conv2D layers, MaxPooling2D layers, Dense layers, and Dropout rates.


**Reasoning**:
I need to construct the CNN model as specified by the subtask instructions. This involves importing TensorFlow Keras layers, building a Sequential model with the given Conv2D, MaxPooling2D, Flatten, Dense, and Dropout layers, and finally printing the model summary.



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Initialize a Sequential model
model = Sequential([
    # First Convolutional Block
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D((2, 2)),

    # Second Convolutional Block
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Flatten the output for the Dense layers
    Flatten(),

    # Dense layers
    Dense(128, activation='relu'),
    Dropout(0.5),

    # Output layer for binary classification
    Dense(1, activation='sigmoid')
])

# Print the model summary
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,625,281 (6.20 MB)

 Trainable params: 1,625,281 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

**Reasoning**:
The CNN model has been successfully defined. The next logical step is to compile the model, which is required before it can be trained. This step will also prepare for varying learning rates as per the overall task requirements.



In [ ]:
import tensorflow as tf

# Compile the model with an optimizer, loss function, and metrics
# Use Adam optimizer with a default learning rate for now, this will be varied later
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("CNN model compiled successfully with Adam optimizer and binary_crossentropy loss.")

CNN model compiled successfully with Adam optimizer and binary_crossentropy loss.


## Train and Evaluate (Original Data)

### Subtask:
For each specified learning rate (e.g., 0.01, 0.005, 0.001), compile and train the CNN model using the *original* training data. Evaluate the model's accuracy on the *testing set* and record the training history (loss and accuracy over epochs).


**Reasoning**:
I need to initialize the `original_data_results` dictionary to store the training history and evaluation metrics for each learning rate, as specified in the instructions for the subtask.



In [ ]:
learning_rates = [0.01, 0.005, 0.001]
original_data_results = {}

print(f"Initialized `learning_rates` to {learning_rates} and `original_data_results` dictionary.")

Initialized `learning_rates` to [0.01, 0.005, 0.001] and `original_data_results` dictionary.


**Reasoning**:
Now that the `learning_rates` and `original_data_results` dictionary are initialized, I need to iterate through each learning rate, define, compile, train, and evaluate a new CNN model instance, and store its history and evaluation metrics in the `original_data_results` dictionary.



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Number of epochs for training
EPOCHS = 10
BATCH_SIZE = 32

for lr in learning_rates:
    print(f"\nTraining model with learning rate: {lr}")

    # 3a. For each iteration, define a new instance of the CNN model
    model_current_lr = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        MaxPooling2D((2, 2)),

        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        # Flatten the output for the Dense layers
        Flatten(),

        # Dense layers
        Dense(128, activation='relu'),
        Dropout(0.5),

        # Output layer for binary classification
        Dense(1, activation='sigmoid')
    ])

    # 3b. Compile the new model instance with the Adam optimizer and specified learning rate
    optimizer = Adam(learning_rate=lr)
    model_current_lr.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # 3c. Train the model using original training data
    history = model_current_lr.fit(
        train_images,
        train_labels,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=1 # Set to 1 to show progress bar, 0 for silent
    )

    # 3d. Evaluate the trained model on the testing set
    test_loss, test_accuracy = model_current_lr.evaluate(test_images, test_labels, verbose=0)
    print(f"Learning Rate: {lr}, Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")

    # 3e. Store the training history and evaluation results
    original_data_results[lr] = {
        'history': history.history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy
    }

print("Finished training and evaluation for all specified learning rates on original data.")


Training model with learning rate: 0.01


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 48s 184ms/step - accuracy: 0.5972 - loss: 1.2706 - val_accuracy: 0.0000e+00 - val_loss: 0.9880
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 80s 176ms/step - accuracy: 0.6272 - loss: 0.6614 - val_accuracy: 0.0000e+00 - val_loss: 0.9489
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 45s 179ms/step - accuracy: 0.6340 - loss: 0.6577 - val_accuracy: 0.0000e+00 - val_loss: 1.0019
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 81s 177ms/step - accuracy: 0.6215 - loss: 0.6641 - val_accuracy: 0.0000e+00 - val_loss: 1.0001
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 44s 177ms/step - accuracy: 0.6115 - loss: 0.6679 - val_accuracy: 0.0000e+00 - val_loss: 1.0165
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 82s 177ms/step - accuracy: 0.6171 - loss: 0.6665 - val_accuracy: 0.0000e+00 - val_loss: 0.9838
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 83s 182ms/step - accuracy: 0.6254 - loss: 0.6614 - val_accuracy: 0.0000e+00 - val_loss: 1.0087
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 47s 187ms/step 

# Task
Initialize a dictionary named `augmented_data_results` to store the training history and evaluation metrics for models trained on augmented data, for each learning rate.

## Initialize Augmented Data Results

### Subtask:
Initialize a dictionary named `augmented_data_results` to store the training history and evaluation metrics for models trained on augmented data, for each learning rate.


**Reasoning**:
I need to initialize an empty dictionary named `augmented_data_results` as specified in the subtask instructions to store results from models trained on augmented data.



In [ ]:
augmented_data_results = {}

print("Initialized 'augmented_data_results' dictionary.")

Initialized 'augmented_data_results' dictionary.


## Train and Evaluate (Augmented Data)

### Subtask:
For each specified learning rate, define a new instance of the CNN model, compile it with the current learning rate, and train it using the combined augmented training data (`combined_augmented_train_images`, `combined_augmented_train_labels`). Evaluate the model's accuracy on the testing set (`test_images`, `test_labels`) and record the training history (loss and accuracy over epochs) and evaluation metrics in the `augmented_data_results` dictionary.


**Reasoning**:
Now that the `augmented_data_results` dictionary is initialized, I need to iterate through each learning rate, define, compile, train, and evaluate a new CNN model instance using the augmented data, and store its history and evaluation metrics in the `augmented_data_results` dictionary.



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Number of epochs for training
EPOCHS = 10
BATCH_SIZE = 32

for lr in learning_rates:
    print(f"\nTraining model with learning rate: {lr} on augmented data")

    # Define a new instance of the CNN model for each learning rate
    model_augmented_lr = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        MaxPooling2D((2, 2)),

        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        # Flatten the output for the Dense layers
        Flatten(),

        # Dense layers
        Dense(128, activation='relu'),
        Dropout(0.5),

        # Output layer for binary classification
        Dense(1, activation='sigmoid')
    ])

    # Compile the new model instance with the Adam optimizer and specified learning rate
    optimizer = Adam(learning_rate=lr)
    model_augmented_lr.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model using augmented training data
    history = model_augmented_lr.fit(
        combined_augmented_train_images,
        combined_augmented_train_labels,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=1 # Set to 1 to show progress bar, 0 for silent
    )

    # Evaluate the trained model on the testing set
    test_loss, test_accuracy = model_augmented_lr.evaluate(test_images, test_labels, verbose=0)
    print(f"Learning Rate: {lr}, Test Accuracy (Augmented Data): {test_accuracy:.4f}, Test Loss (Augmented Data): {test_loss:.4f}")

    # Store the training history and evaluation results
    augmented_data_results[lr] = {
        'history': history.history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy
    }

print("Finished training and evaluation for all specified learning rates on augmented data.")

NameError: name 'learning_rates' is not defined

**Reasoning**:
The previous code failed because the `learning_rates` variable was not defined in the current execution context. I need to re-define `learning_rates` within this code block to ensure it is accessible.



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define learning rates
learning_rates = [0.01, 0.005, 0.001]

# Number of epochs for training
EPOCHS = 10
BATCH_SIZE = 32

for lr in learning_rates:
    print(f"\nTraining model with learning rate: {lr} on augmented data")

    # Define a new instance of the CNN model for each learning rate
    model_augmented_lr = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        MaxPooling2D((2, 2)),

        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        # Flatten the output for the Dense layers
        Flatten(),

        # Dense layers
        Dense(128, activation='relu'),
        Dropout(0.5),

        # Output layer for binary classification
        Dense(1, activation='sigmoid')
    ])

    # Compile the new model instance with the Adam optimizer and specified learning rate
    optimizer = Adam(learning_rate=lr)
    model_augmented_lr.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model using augmented training data
    history = model_augmented_lr.fit(
        combined_augmented_train_images,
        combined_augmented_train_labels,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=1 # Set to 1 to show progress bar, 0 for silent
    )

    # Evaluate the trained model on the testing set
    test_loss, test_accuracy = model_augmented_lr.evaluate(test_images, test_labels, verbose=0)
    print(f"Learning Rate: {lr}, Test Accuracy (Augmented Data): {test_accuracy:.4f}, Test Loss (Augmented Data): {test_loss:.4f}")

    # Store the training history and evaluation results
    augmented_data_results[lr] = {
        'history': history.history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy
    }

print("Finished training and evaluation for all specified learning rates on augmented data.")


Training model with learning rate: 0.01 on augmented data


NameError: name 'IMG_HEIGHT' is not defined

**Reasoning**:
The previous code failed because `IMG_HEIGHT` and `IMG_WIDTH` were not defined in the current execution context. I need to re-define these variables within this code block to ensure they are accessible for the model definition.



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define learning rates
learning_rates = [0.01, 0.005, 0.001]

# Define image dimensions (re-defining them as they were lost in the previous execution context)
IMG_HEIGHT = 64
IMG_WIDTH = 64

# Number of epochs for training
EPOCHS = 10
BATCH_SIZE = 32

for lr in learning_rates:
    print(f"\nTraining model with learning rate: {lr} on augmented data")

    # Define a new instance of the CNN model for each learning rate
    model_augmented_lr = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        MaxPooling2D((2, 2)),

        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        # Flatten the output for the Dense layers
        Flatten(),

        # Dense layers
        Dense(128, activation='relu'),
        Dropout(0.5),

        # Output layer for binary classification
        Dense(1, activation='sigmoid')
    ])

    # Compile the new model instance with the Adam optimizer and specified learning rate
    optimizer = Adam(learning_rate=lr)
    model_augmented_lr.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model using augmented training data
    history = model_augmented_lr.fit(
        combined_augmented_train_images,
        combined_augmented_train_labels,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=1 # Set to 1 to show progress bar, 0 for silent
    )

    # Evaluate the trained model on the testing set
    test_loss, test_accuracy = model_augmented_lr.evaluate(test_images, test_labels, verbose=0)
    print(f"Learning Rate: {lr}, Test Accuracy (Augmented Data): {test_accuracy:.4f}, Test Loss (Augmented Data): {test_loss:.4f}")

    # Store the training history and evaluation results
    augmented_data_results[lr] = {
        'history': history.history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy
    }

print("Finished training and evaluation for all specified learning rates on augmented data.")


Training model with learning rate: 0.01 on augmented data


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'combined_augmented_train_images' is not defined

**Reasoning**:
The previous code failed because several critical variables and functions, including data loading, preprocessing, and augmentation functions, along with the `combined_augmented_train_images` and `combined_augmented_train_labels` arrays, were not defined in the current execution context. To resolve this, I need to include all necessary data loading, preprocessing, augmentation function definitions, and data array creations within this single code block to ensure the model training can proceed without `NameError`.



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import os
from skimage.transform import rotate

# Define learning rates
learning_rates = [0.01, 0.005, 0.001]

# Define image dimensions
IMG_HEIGHT = 64
IMG_WIDTH = 64

# Number of epochs for training
EPOCHS = 10
BATCH_SIZE = 32

# --- Data Loading and Preprocessing (from previous steps) ---
# Define the base directory where content was extracted
base_dir = '/content/cat_dog/'

# Define paths for training and testing images and labels
train_images_dir = os.path.join(base_dir, 'train')
train_labels_file = os.path.join(base_dir, 'train_class_labels.txt')
test_images_dir = os.path.join(base_dir, 'test')
test_labels_file = os.path.join(base_dir, 'test_class_labels.txt')

def load_images_from_directory(directory, target_size=(IMG_HEIGHT, IMG_WIDTH)):
    images = []
    image_filenames = sorted([f for f in os.listdir(directory) if f.endswith(('.jpg', '.png'))])
    for filename in image_filenames:
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path) # Reads image in BGR format
        if img is not None:
            img = cv2.resize(img, target_size) # Resize image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR to RGB
            images.append(img)
    return images

def load_labels_from_file(filepath):
    labels = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            labels.append(int(parts[-1]))
    return labels

# Load training data
train_images = load_images_from_directory(train_images_dir)
train_labels = load_labels_from_file(train_labels_file)

# Load testing data
test_images = load_images_from_directory(test_images_dir)
test_labels = load_labels_from_file(test_labels_file)

# Convert to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize pixel values to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# --- Custom Data Augmentation Function (from previous steps) ---
def augment_image_rotation(image, label):
    augmented_images = []
    augmented_labels = []

    angle1 = np.random.uniform(-45, 45)
    angle2 = np.random.uniform(-45, 45)

    rotated_image1 = rotate(image, angle=angle1, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image1)
    augmented_labels.append(label)

    rotated_image2 = rotate(image, angle=angle2, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image2)
    augmented_labels.append(label)

    return augmented_images, augmented_labels

# --- Generate Augmented Training Dataset (from previous steps) ---
augmented_train_images_list = []
augmented_train_labels_list = []

for i in range(len(train_images)):
    image = train_images[i]
    label = train_labels[i]

    aug_imgs, aug_labels = augment_image_rotation(image, label)

    augmented_train_images_list.extend(aug_imgs);
    augmented_train_labels_list.extend(aug_labels);

augmented_train_images = np.array(augmented_train_images_list)
augmented_train_labels = np.array(augmented_train_labels_list)

combined_augmented_train_images = np.concatenate((train_images, augmented_train_images), axis=0)
combined_augmented_train_labels = np.concatenate((train_labels, augmented_train_labels), axis=0)

# --- Training and Evaluation Loop for Augmented Data ---
for lr in learning_rates:
    print(f"\nTraining model with learning rate: {lr} on augmented data")

    # Define a new instance of the CNN model for each learning rate
    model_augmented_lr = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        MaxPooling2D((2, 2)),

        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        # Flatten the output for the Dense layers
        Flatten(),

        # Dense layers
        Dense(128, activation='relu'),
        Dropout(0.5),

        # Output layer for binary classification
        Dense(1, activation='sigmoid')
    ])

    # Compile the new model instance with the Adam optimizer and specified learning rate
    optimizer = Adam(learning_rate=lr)
    model_augmented_lr.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model using augmented training data
    history = model_augmented_lr.fit(
        combined_augmented_train_images,
        combined_augmented_train_labels,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=1
    )

    # Evaluate the trained model on the testing set
    test_loss, test_accuracy = model_augmented_lr.evaluate(test_images, test_labels, verbose=0)
    print(f"Learning Rate: {lr}, Test Accuracy (Augmented Data): {test_accuracy:.4f}, Test Loss (Augmented Data): {test_loss:.4f}")

    # Store the training history and evaluation results
    augmented_data_results[lr] = {
        'history': history.history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy
    }

print("Finished training and evaluation for all specified learning rates on augmented data.")


Training model with learning rate: 0.01 on augmented data


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Reasoning**:
The previous code failed because `augmented_data_results` was not defined in the current execution context. I need to re-initialize this dictionary within this code block to ensure it is accessible for storing the training and evaluation results.



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import os
from skimage.transform import rotate

# Define learning rates
learning_rates = [0.01, 0.005, 0.001]

# Define image dimensions
IMG_HEIGHT = 64
IMG_WIDTH = 64

# Number of epochs for training
EPOCHS = 10
BATCH_SIZE = 32

# Initialize augmented_data_results dictionary
augmented_data_results = {}

# --- Data Loading and Preprocessing (from previous steps) ---
# Define the base directory where content was extracted
base_dir = '/content/cat_dog/'

# Define paths for training and testing images and labels
train_images_dir = os.path.join(base_dir, 'train')
train_labels_file = os.path.join(base_dir, 'train_class_labels.txt')
test_images_dir = os.path.join(base_dir, 'test')
test_labels_file = os.path.join(base_dir, 'test_class_labels.txt')

def load_images_from_directory(directory, target_size=(IMG_HEIGHT, IMG_WIDTH)):
    images = []
    image_filenames = sorted([f for f in os.listdir(directory) if f.endswith(('.jpg', '.png'))])
    for filename in image_filenames:
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path) # Reads image in BGR format
        if img is not None:
            img = cv2.resize(img, target_size) # Resize image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR to RGB
            images.append(img)
    return images

def load_labels_from_file(filepath):
    labels = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            labels.append(int(parts[-1]))
    return labels

# Load training data
train_images = load_images_from_directory(train_images_dir)
train_labels = load_labels_from_file(train_labels_file)

# Load testing data
test_images = load_images_from_directory(test_images_dir)
test_labels = load_labels_from_file(test_labels_file)

# Convert to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize pixel values to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# --- Custom Data Augmentation Function (from previous steps) ---
def augment_image_rotation(image, label):
    augmented_images = []
    augmented_labels = []

    angle1 = np.random.uniform(-45, 45)
    angle2 = np.random.uniform(-45, 45)

    rotated_image1 = rotate(image, angle=angle1, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image1)
    augmented_labels.append(label)

    rotated_image2 = rotate(image, angle=angle2, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image2)
    augmented_labels.append(label)

    return augmented_images, augmented_labels

# --- Generate Augmented Training Dataset (from previous steps) ---
augmented_train_images_list = []
augmented_train_labels_list = []

for i in range(len(train_images)):
    image = train_images[i]
    label = train_labels[i]

    aug_imgs, aug_labels = augment_image_rotation(image, label)

    augmented_train_images_list.extend(aug_imgs);
    augmented_train_labels_list.extend(aug_labels);

augmented_train_images = np.array(augmented_train_images_list)
augmented_train_labels = np.array(augmented_train_labels_list)

combined_augmented_train_images = np.concatenate((train_images, augmented_train_images), axis=0)
combined_augmented_train_labels = np.concatenate((train_labels, augmented_train_labels), axis=0)

# --- Training and Evaluation Loop for Augmented Data ---
for lr in learning_rates:
    print(f"\nTraining model with learning rate: {lr} on augmented data")

    # Define a new instance of the CNN model for each learning rate
    model_augmented_lr = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        MaxPooling2D((2, 2)),

        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        # Flatten the output for the Dense layers
        Flatten(),

        # Dense layers
        Dense(128, activation='relu'),
        Dropout(0.5),

        # Output layer for binary classification
        Dense(1, activation='sigmoid')
    ])

    # Compile the new model instance with the Adam optimizer and specified learning rate
    optimizer = Adam(learning_rate=lr)
    model_augmented_lr.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model using augmented training data
    history = model_augmented_lr.fit(
        combined_augmented_train_images,
        combined_augmented_train_labels,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=1
    )

    # Evaluate the trained model on the testing set
    test_loss, test_accuracy = model_augmented_lr.evaluate(test_images, test_labels, verbose=0)
    print(f"Learning Rate: {lr}, Test Accuracy (Augmented Data): {test_accuracy:.4f}, Test Loss (Augmented Data): {test_loss:.4f}")

    # Store the training history and evaluation results
    augmented_data_results[lr] = {
        'history': history.history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy
    }

print("Finished training and evaluation for all specified learning rates on augmented data.")


Training model with learning rate: 0.01 on augmented data


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Reasoning**:
The previous code block successfully executed without a runtime error, continuing the training and evaluation process for models on augmented data across different learning rates. The next logical step in the overall task is to summarize the results from both original and augmented data training in a table, as specified in the main task description.



In [ ]:
## Summarize Results in a Table

### Subtask:
Create a summary table that compares the test accuracy and test loss for each learning rate when training on original data versus augmented data. The table should clearly show the learning rate, data type (original/augmented), test accuracy, and test loss.

SyntaxError: invalid syntax (ipython-input-628637306.py, line 4)

**Reasoning**:
The previous step defined the subtask of summarizing the model's performance. Now, I will generate the Python code to create a summary table by extracting the test accuracy and test loss for each learning rate from both the `original_data_results` and `augmented_data_results` dictionaries, and then present this information using a pandas DataFrame.



In [ ]:
import pandas as pd

summary_data = []

# Process original data results
for lr in learning_rates:
    result = original_data_results[lr]
    summary_data.append({
        'Learning Rate': lr,
        'Data Type': 'Original',
        'Test Accuracy': result['test_accuracy'],
        'Test Loss': result['test_loss']
    })

# Process augmented data results
for lr in learning_rates:
    result = augmented_data_results[lr]
    summary_data.append({
        'Learning Rate': lr,
        'Data Type': 'Augmented',
        'Test Accuracy': result['test_accuracy'],
        'Test Loss': result['test_loss']
    })

summary_df = pd.DataFrame(summary_data)
print("Summary of Model Performance:")
print(summary_df.to_string(index=False))

NameError: name 'learning_rates' is not defined

**Reasoning**:
The previous code failed because the `learning_rates`, `original_data_results`, and `augmented_data_results` variables were not defined in the current execution context. I need to re-initialize all necessary variables and re-run the previous training and evaluation loops to ensure all data and results are available for generating the summary table.



In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import os
from skimage.transform import rotate

# Define learning rates
learning_rates = [0.01, 0.005, 0.001]

# Define image dimensions
IMG_HEIGHT = 64
IMG_WIDTH = 64

# Number of epochs for training
EPOCHS = 10
BATCH_SIZE = 32

# Initialize result dictionaries
original_data_results = {}
augmented_data_results = {}

# --- Data Loading and Preprocessing (from previous steps) ---
# Define the base directory where content was extracted
base_dir = '/content/cat_dog/'

# Define paths for training and testing images and labels
train_images_dir = os.path.join(base_dir, 'train')
train_labels_file = os.path.join(base_dir, 'train_class_labels.txt')
test_images_dir = os.path.join(base_dir, 'test')
test_labels_file = os.path.join(base_dir, 'test_class_labels.txt')

def load_images_from_directory(directory, target_size=(IMG_HEIGHT, IMG_WIDTH)):
    images = []
    image_filenames = sorted([f for f in os.listdir(directory) if f.endswith(('.jpg', '.png'))])
    for filename in image_filenames:
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path) # Reads image in BGR format
        if img is not None:
            img = cv2.resize(img, target_size) # Resize image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR to RGB
            images.append(img)
    return images

def load_labels_from_file(filepath):
    labels = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            labels.append(int(parts[-1]))
    return labels

# Load training data
train_images = load_images_from_directory(train_images_dir)
train_labels = load_labels_from_file(train_labels_file)

# Load testing data
test_images = load_images_from_directory(test_images_dir)
test_labels = load_labels_from_file(test_labels_file)

# Convert to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize pixel values to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# --- Custom Data Augmentation Function (from previous steps) ---
def augment_image_rotation(image, label):
    augmented_images = []
    augmented_labels = []

    angle1 = np.random.uniform(-45, 45)
    angle2 = np.random.uniform(-45, 45)

    rotated_image1 = rotate(image, angle=angle1, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image1)
    augmented_labels.append(label)

    rotated_image2 = rotate(image, angle=angle2, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image2)
    augmented_labels.append(label)

    return augmented_images, augmented_labels

# --- Generate Augmented Training Dataset (from previous steps) ---
augmented_train_images_list = []
augmented_train_labels_list = []

for i in range(len(train_images)):
    image = train_images[i]
    label = train_labels[i]

    aug_imgs, aug_labels = augment_image_rotation(image, label)

    augmented_train_images_list.extend(aug_imgs);
    augmented_train_labels_list.extend(aug_labels);

augmented_train_images = np.array(augmented_train_images_list)
augmented_train_labels = np.array(augmented_train_labels_list)

combined_augmented_train_images = np.concatenate((train_images, augmented_train_images), axis=0)
combined_augmented_train_labels = np.concatenate((train_labels, augmented_train_labels), axis=0)

# --- Training and Evaluation Loop for Original Data (re-run to populate original_data_results) ---
print("\n--- Training on Original Data ---")
for lr in learning_rates:
    print(f"\nTraining model with learning rate: {lr}")

    model_original_lr = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    optimizer = Adam(learning_rate=lr)
    model_original_lr.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    history = model_original_lr.fit(
        train_images,
        train_labels,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=0
    )

    test_loss, test_accuracy = model_original_lr.evaluate(test_images, test_labels, verbose=0)
    print(f"Learning Rate: {lr}, Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")

    original_data_results[lr] = {
        'history': history.history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy
    }
print("Finished training and evaluation for all specified learning rates on original data.")

# --- Training and Evaluation Loop for Augmented Data (re-run to populate augmented_data_results) ---
print("\n--- Training on Augmented Data ---")
for lr in learning_rates:
    print(f"\nTraining model with learning rate: {lr} on augmented data")

    model_augmented_lr = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    optimizer = Adam(learning_rate=lr)
    model_augmented_lr.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    history = model_augmented_lr.fit(
        combined_augmented_train_images,
        combined_augmented_train_labels,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=0
    )

    test_loss, test_accuracy = model_augmented_lr.evaluate(test_images, test_labels, verbose=0)
    print(f"Learning Rate: {lr}, Test Accuracy (Augmented Data): {test_accuracy:.4f}, Test Loss (Augmented Data): {test_loss:.4f}")

    augmented_data_results[lr] = {
        'history': history.history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy
    }
print("Finished training and evaluation for all specified learning rates on augmented data.")

# --- Summary Table Generation ---
summary_data = []

# Process original data results
for lr in learning_rates:
    result = original_data_results[lr]
    summary_data.append({
        'Learning Rate': lr,
        'Data Type': 'Original',
        'Test Accuracy': result['test_accuracy'],
        'Test Loss': result['test_loss']
    })

# Process augmented data results
for lr in learning_rates:
    result = augmented_data_results[lr]
    summary_data.append({
        'Learning Rate': lr,
        'Data Type': 'Augmented',
        'Test Accuracy': result['test_accuracy'],
        'Test Loss': result['test_loss']
    })

summary_df = pd.DataFrame(summary_data)
print("\nSummary of Model Performance:")
print(summary_df.to_string(index=False))


--- Training on Original Data ---

Training model with learning rate: 0.01


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Learning Rate: 0.01, Test Accuracy: 0.5000, Test Loss: 0.7206

Training model with learning rate: 0.005


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Learning Rate: 0.005, Test Accuracy: 0.5000, Test Loss: 0.7281

Training model with learning rate: 0.001


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Learning Rate: 0.001, Test Accuracy: 0.4935, Test Loss: 0.7594
Finished training and evaluation for all specified learning rates on original data.

--- Training on Augmented Data ---

Training model with learning rate: 0.01 on augmented data


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Task
The execution of the current code cell is still in progress. Please wait for it to complete.

## Initialize Environment and Data

### Subtask:
Re-execute all necessary imports, define global constants (IMG_HEIGHT, IMG_WIDTH, EPOCHS, BATCH_SIZE, learning_rates), and re-load/preprocess the image data (train_images, train_labels, test_images, test_labels). Also, redefine the `augment_image_rotation` function and regenerate the `combined_augmented_train_images` and `combined_augmented_train_labels` datasets. Initialize `original_data_results` and `augmented_data_results` dictionaries.


**Reasoning**:
To ensure all necessary variables, functions, and data are correctly initialized and available for subsequent steps, I will re-execute all imports, define global constants, load and preprocess data, redefine the data augmentation function, generate the augmented dataset, and initialize the results dictionaries as specified in the subtask.



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import os
from skimage.transform import rotate
import pandas as pd

# 2. Define the global constants
IMG_HEIGHT = 64
IMG_WIDTH = 64
EPOCHS = 10
BATCH_SIZE = 32
learning_rates = [0.01, 0.005, 0.001]

# 3. Define the base directory for data extraction
base_dir = '/content/cat_dog/'

# Define paths for training and testing images and labels
train_images_dir = os.path.join(base_dir, 'train')
train_labels_file = os.path.join(base_dir, 'train_class_labels.txt')
test_images_dir = os.path.join(base_dir, 'test')
test_labels_file = os.path.join(base_dir, 'test_class_labels.txt')

# 4. Define helper functions
def load_images_from_directory(directory, target_size=(IMG_HEIGHT, IMG_WIDTH)):
    images = []
    image_filenames = sorted([f for f in os.listdir(directory) if f.endswith(('.jpg', '.png'))])
    for filename in image_filenames:
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path) # Reads image in BGR format
        if img is not None:
            img = cv2.resize(img, target_size) # Resize image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR to RGB
            images.append(img)
    return images

def load_labels_from_file(filepath):
    labels = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            labels.append(int(parts[-1]))
    return labels

# 5. Load and preprocess the image data
train_images = load_images_from_directory(train_images_dir)
train_labels = load_labels_from_file(train_labels_file)
test_images = load_images_from_directory(test_images_dir)
test_labels = load_labels_from_file(test_labels_file)

# Convert to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize pixel values to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# 6. Redefine the `augment_image_rotation` function
def augment_image_rotation(image, label):
    augmented_images = []
    augmented_labels = []

    angle1 = np.random.uniform(-45, 45)
    angle2 = np.random.uniform(-45, 45)

    rotated_image1 = rotate(image, angle=angle1, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image1)
    augmented_labels.append(label)

    rotated_image2 = rotate(image, angle=angle2, resize=False, preserve_range=True, mode='edge')
    augmented_images.append(rotated_image2)
    augmented_labels.append(label)

    return augmented_images, augmented_labels

# 7. Generate the augmented training dataset
augmented_train_images_list = []
augmented_train_labels_list = []

for i in range(len(train_images)):
    image = train_images[i]
    label = train_labels[i]

    aug_imgs, aug_labels = augment_image_rotation(image, label)

    augmented_train_images_list.extend(aug_imgs);
    augmented_train_labels_list.extend(aug_labels);

augmented_train_images = np.array(augmented_train_images_list)
augmented_train_labels = np.array(augmented_train_labels_list)

combined_augmented_train_images = np.concatenate((train_images, augmented_train_images), axis=0)
combined_augmented_train_labels = np.concatenate((train_labels, augmented_train_labels), axis=0)

# 8. Initialize result dictionaries
original_data_results = {}
augmented_data_results = {}

print("Environment and data initialized successfully.")
print(f"train_images shape: {train_images.shape}, train_labels shape: {train_labels.shape}")
print(f"test_images shape: {test_images.shape}, test_labels shape: {test_labels.shape}")
print(f"combined_augmented_train_images shape: {combined_augmented_train_images.shape}")
print(f"combined_augmented_train_labels shape: {combined_augmented_train_labels.shape}")

# Task
The execution of the current code cell has been approved.

Next, I'll proceed with the following step as per the plan:
**Define CNN Model Template**: Define a function or class to construct the Convolutional Neural Network (CNN) model as per the updated architecture specifications: Layer 1: Conv2D(32, (3,3), activation='relu'). Layer 2: MaxPooling2D((2,2)) followed by Dropout(rate=0.2). Layer 3: Conv2D(64, (3,3), activation='relu'). Layer 4: MaxPooling2D((2,2)) followed by Dropout(rate=0.3). Layer 5: Dense(100, activation='relu') followed by Dropout(rate=0.5). Layer 6: Output Dense(1, activation='sigmoid').

## Define CNN Model Template

### Subtask:
Define a function or class to construct the Convolutional Neural Network (CNN) model as per the updated architecture specifications.


**Reasoning**:
I need to define a Python function `create_cnn_model` that encapsulates the CNN architecture as specified in the subtask. This involves using `tensorflow.keras.models.Sequential` and adding the `Conv2D`, `MaxPooling2D`, `Dropout`, `Flatten`, and `Dense` layers with the given parameters.



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_cnn_model(input_shape):
    model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.3),

        # Flatten the output for the Dense layers
        Flatten(),

        # Dense layers
        Dense(100, activation='relu'),
        Dropout(0.5),

        # Output layer for binary classification
        Dense(1, activation='sigmoid')
    ])
    return model

print("Defined 'create_cnn_model' function for constructing the CNN model.")